In [1]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


What is generative AI?

Generative Ai is a class of artificial Inteligence technique and Algorithm designed to generate a new content or ouput or data 

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [71]:
os.environ["HF_KEY"]=os.getenv("HF_KEY")
os.environ["GOOGLE_API_KEY"]=os.getenv("GOOGLE_API_KEY")

In [3]:
url_load=WebBaseLoader("https://seths.blog/page/2/")

In [44]:
docs=url_load.load()

In [50]:
type(docs)

list

In [47]:
#Text splitter

from langchain_text_splitters import RecursiveCharacterTextSplitter,CharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(chunk_size=50,chunk_overlap=0)

In [52]:
split_docs=splitter.split_documents(docs)

In [55]:
split_docs

[Document(metadata={'source': 'https://seths.blog/page/2/', 'title': "Seth's Blog", 'description': "Seth Godin's Blog on marketing, tribes and respect", 'language': 'en'}, page_content="Seth's Blog"),
 Document(metadata={'source': 'https://seths.blog/page/2/', 'title': "Seth's Blog", 'description': "Seth Godin's Blog on marketing, tribes and respect", 'language': 'en'}, page_content='You have unread updates\n0'),
 Document(metadata={'source': 'https://seths.blog/page/2/', 'title': "Seth's Blog", 'description': "Seth Godin's Blog on marketing, tribes and respect", 'language': 'en'}, page_content='Search\n\n\nSearch for:'),
 Document(metadata={'source': 'https://seths.blog/page/2/', 'title': "Seth's Blog", 'description': "Seth Godin's Blog on marketing, tribes and respect", 'language': 'en'}, page_content='Or try my new AI-powered search bot\nSubscribe'),
 Document(metadata={'source': 'https://seths.blog/page/2/', 'title': "Seth's Blog", 'description': "Seth Godin's Blog on marketing, tr

In [56]:
print(split_docs[0])
print(split_docs[1])

page_content='Seth's Blog' metadata={'source': 'https://seths.blog/page/2/', 'title': "Seth's Blog", 'description': "Seth Godin's Blog on marketing, tribes and respect", 'language': 'en'}
page_content='You have unread updates
0' metadata={'source': 'https://seths.blog/page/2/', 'title': "Seth's Blog", 'description': "Seth Godin's Blog on marketing, tribes and respect", 'language': 'en'}


In [57]:
#HuggingFace Embeddings
from langchain_huggingface import HuggingFaceEmbeddings
embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [58]:
#vector store DB
from langchain_community.vectorstores import chroma,FAISS

In [60]:
vectorDB=FAISS.from_documents(split_docs,embedding=embedding)

In [62]:
db=vectorDB.similarity_search("Who made a movie?")

In [67]:
db[0].page_content

'space based.'

In [74]:
from langchain_core.prompts import ChatPromptTemplate,PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

In [82]:
prompts=ChatPromptTemplate.from_template(
    """Answer the following question smartly
   <context>
    {context}
    </context>"
    """
    
)

In [78]:
#initializing llm model
from langchain_google_genai import ChatGoogleGenerativeAI

In [72]:
llm=ChatGoogleGenerativeAI(model="gemini-1.5-pro")

In [83]:
#combine document
document=create_stuff_documents_chain(prompt=prompts,llm=llm)

In [84]:
document

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='Answer the following question smartly\n   <context>\n    {context}\n    </context>"\n    '), additional_kwargs={})])
| ChatGoogleGenerativeAI(model='models/gemini-1.5-pro', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002798872B340>, default_metadata=())
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [87]:
#initialization of retriever
#input ----> retriever ---> vectorDB
retriever=vectorDB.as_retriever()

In [88]:
from langchain.chains.retrieval import create_retrieval_chain

In [89]:
retriever_chain=create_retrieval_chain(retriever,document)

In [90]:
retriever_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000027984C22B90>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='Answer the following question smartly\n   <context>\n    {context}\n    </context>"\n    '), additional_kwargs={})])
            | ChatGoogleGen